## About this notebook...
A/c to me, this is one of the best competition on kaggle. It give you a chance to learn some cool new things on kaggle and broad you knowledge in Datascience.

As already written in Introduction, Human brain research is among the most complex areas of study for scientists. We know that age and other factors can affect its function and structure, but more research is needed into what specifically occurs within the brain. With much of the research using MRI scans, data scientists are well positioned to support future insights. In particular, neuroimaging specialists look for measurable markers of behavior, health, or disorder to help identify relevant brain regions and their contribution to typical or symptomatic effects.

I try my level best to give all things as simple as possible.

# Understand the Problem statment!!!

# What is NeuroImaging?
Neuroimaging or brain imaging is the use of various techniques to either directly or indirectly image the structure, function, or pharmacology of the nervous system.It is a relatively new discipline within medicine, neuroscience, and psychology.Physicians who specialize in the performance and interpretation of neuroimaging in the clinical setting are neuroradiologists.

Neuroimaging falls into two broad categories:

Structural imaging, which deals with the structure of the nervous system and the diagnosis of gross (large scale) intracranial disease (such as a tumor) and injury.
Functional imaging, which is used to diagnose metabolic diseases and lesions on a finer scale (such as Alzheimer's disease) and also for neurological and cognitive psychology research and building brain-computer interfaces.
Human brain research is among the most complex areas of study for scientists. We know that age and other factors can affect its function and structure, but more research is needed into what specifically occurs within the brain. With much of the research using MRI scans, data scientists are well positioned to support future insights. In particular, neuroimaging specialists look for measurable markers of behavior, health, or disorder to help identify relevant brain regions and their contribution to typical or symptomatic effects.

**In this challenge, we have to predict age and assessment values from two domains using features derived from brain MRI images as inputs.**

# How Brain MRI is done?
Magnetic resonance imaging (MRI) of the head is a painless, noninvasive test that produces detailed images of your brain and brain stem. An MRI machine creates the images using a magnetic field and radio waves. This test is also known as a brain MRI or a cranial MRI. You will go to a hospital or radiology center to take a head MRI.

An MRI scan is different from a CT scan or an X-ray in that it doesn’t use radiation to produce images. An MRI scan combines images to create a 3-D picture of your internal structures, so it’s more effective than other scans at detecting abnormalities in small structures of the brain such as the pituitary gland and brain stem. Sometimes a contrast agent, or dye, can be given through an intravenous (IV) line to better visualize certain structures or abnormalities.

A functional MRI (fMRI) of the brain is useful for people who might have to undergo brain surgery. An fMRI can pinpoint areas of the brain responsible for speech and language, and body movement. It does this by measuring metabolic changes that take place in your brain when you perform certain tasks. During this test, you may need to carry out small tasks, such as answering basic questions or tapping your thumb with your fingertips.

## Let understand the Dataset.
* fMRI_train - a folder containing 53 3D spatial maps for train samples in [.mat] format.
* fMRI_test - a folder containing 53 3D spatial maps for test samples in [.mat] format.
* fnc.csv - static FNC correlation features for both train and test samples.
* loading.csv - sMRI SBM loadings for both train and test samples.
* train_scores.csv - age and assessment values for train samples.
* reveal_ID_site2.csv - a list of subject IDs whose data was collected with a different scanner than the train samples.
* fMRI_mask.nii - a 3D binary spatial map.
* ICN_numbers.txt - intrinsic connectivity network numbers for each fMRI spatial map; matches FNC names.


# How Features Were Obtained
An unbiased strategy was utilized to obtain the provided features. This means that a separate, unrelated large imaging dataset was utilized to learn feature templates. Then, these templates were "projected" onto the original imaging data of each subject used for this competition using spatially constrained independent component analysis (scICA) via group information guided ICA (GIG-ICA).

The first set of features are source-based morphometry (SBM) loadings. These are subject-level weights from a group-level ICA decomposition of gray matter concentration maps from structural MRI (sMRI) scans.

The second set are static functional network connectivity (FNC) matrices. These are the subject-level cross-correlation values among 53 component timecourses estimated from GIG-ICA of resting state functional MRI (fMRI).

The third set of features are the component spatial maps (SM). These are the subject-level 3D images of 53 spatial networks estimated from GIG-ICA of resting state functional MRI (fMRI).

In [ ]:
!wget https://github.com/Chaogan-Yan/DPABI/raw/master/Templates/ch2better.nii

## Load Necessary Library for this project

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import h5py

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from scipy.stats import ks_2samp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import lightgbm as lgb

import nilearn as nl
import nilearn.plotting as nlplt
import nibabel as nib

SEED = 1337

### Load Data

In [ ]:
df_fnc = pd.read_csv('../input/trends-assessment-prediction/fnc.csv')
df_loading = pd.read_csv('../input/trends-assessment-prediction/loading.csv')
df_train_scores = pd.read_csv('../input/trends-assessment-prediction/train_scores.csv')

fnc_features, loading_features = list(df_fnc.columns[1:]), list(df_loading.columns[1:])
df_train_scores['is_train'] = 1
df = df_fnc.merge(df_loading, on='Id')
df = df.merge(df_train_scores, how='left', on='Id')

df.loc[df['is_train'].isnull(), 'is_train'] = 0
df['is_train'] = df['is_train'].astype(np.uint8)
df['Id'] = df['Id'].astype(np.uint16)

print(f'Static FNC Correlation Shape = {df_fnc.shape}')
print(f'Static FNC Correlation Memory Usage = {df_fnc.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'sMRI SBM Loadings Shape = {df_loading.shape}')
print(f'sMRI SBM Loadings Memory Usage = {df_loading.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Train Scores Shape = {df_train_scores.shape}')
print(f'Train Scores Memory Usage = {df_train_scores.memory_usage().sum() / 1024 ** 2:.2f} MB')
print('-------------------------------------')
print(f'Train & Test Set Shape = {df.shape}')
print(f'Train & Test Set Memory Usage = {df.memory_usage().sum() / 1024 ** 2:.2f} MB')

del df_fnc, df_loading, df_train_scores

## **1. Train Scores (Targets)**
`train_scores.csv` is the file which consists of target features. Those features are `age`, `domain1_var1`, `domain1_var2`, `domain2_var1` and `domain2_var2`. There are 5 target features to predict and submissions are scored using feature-weighted, normalized absolute errors.

**score** ${= \Large \sum\limits_{f} w_{f} \Big( \frac{\sum_{i} \big| y_{f,i}  - \hat{y}_{f,i}\big| }{\sum_{i} \hat{y}_{f,i}} \Big) }$

The weights are `[.3, .175, .175, .175, .175]` corresponding to features `[age, domain1_var1, domain1_var2, domain2_var1, domain2_var2]`. This means every targets normalized absolute error is independent from each other. They can be trained and predicted with a single model or 5 different models.

Another important thing to consider is, `train_scores.csv` are not the original age and raw assessment values. They have been transformed and de-identified to help protect subject identity and minimize the risk of unethical usage of the data. Nonetheless, they are directly derived from the original assessment values and, thus, associations with the provided features is equally likely.

**Before transformation, the age in the training set is rounded to nearest year for privacy reasons. However, age is not rounded to year (higher precision) in the test set. Thus, heavily overfitting to the training set age will very likely have a negative impact on your submissions.**

### **1.1 Targets' Distributions**

Even though some of target features are slightly tailed, all of them follow a normal distribution. Their descriptive statistical summary are very similar to each other.

A small percentage of values are missing in target features except `age`. Target features in the same domain have same number of missing values and they are missing in same samples.  Those are skipped in the score calculation. However, every row should be predicted in the submission file.

In [ ]:
def plot_target(target_feature):   
    
    if target_feature == 'age':
        print(f'Target feature {target_feature} Statistical Analysis\n{"-" * 39}')
    else:
        print(f'Target feature {target_feature} Statistical Analysis\n{"-" * 48}')
        
    print(f'Mean: {df[target_feature].mean():.4}  -  Median: {df[target_feature].median():.4}  -  Std: {df[target_feature].std():.4}')
    print(f'Min: {df[target_feature].min():.4}  -  25%: {df[target_feature].quantile(0.25):.4}  -  50%: {df[target_feature].quantile(0.5):.4}  -  75%: {df[target_feature].quantile(0.75):.4}  -  Max: {df[target_feature].max():.4}')
    print(f'Skew: {df[target_feature].skew():.4}  -  Kurtosis: {df[target_feature].kurtosis():.4}')
    missing_values_count = df[(df['is_train'] == 1) & (df[target_feature]).isnull()].shape[0]
    training_samples_count = df[df['is_train'] == 1].shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig = plt.subplots(figsize=(6, 4), dpi=100)

    sns.distplot(df[target_feature], label=target_feature)

    plt.xlabel('')
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    plt.title(f'{target_feature} Distribution in Training Set')
    plt.show()
    
for target_feature in ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']:
    plot_target(target_feature)

### **1.2 Targets' Correlations**

Target features are not correlated with each other too much. The strongest correlations are between `age` and `domain1_var1` (**0.34**), and between  `age` and `domain2_var1` (**0.23**). There might be a relationship between  `age` and var1 features.

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)

sns.heatmap(df[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 14}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=14, rotation=45)
plt.tick_params(axis='y', labelsize=14, rotation=0)
    
plt.title('Target Features Correlations', size=18, pad=18)
plt.show()

## **2. Source-based Morphometry Loadings**
The first set of features are source-based morphometry (SBM) loadings. These are subject-level weights from a group-level ICA decomposition of gray matter concentration maps from structural MRI (sMRI) scans. Those features are for both training and test samples.

There are **26** features in `loading.csv` file without `Id`. Those features are named from `IC_01` to `IC_30`, but `IC_19`, `IC_23`, `IC_25` and `IC_27` don't exist. They are parts of brain and their explanations are listed below:

`
IC_01 - Cerebellum
IC_02 - ACC+mpfc
IC_03 - Caudate
IC_04 - Cerebellum
IC_05 - Calcarine
IC_06 - Calcarine
IC_07 - Precuneus+PCC
IC_08 - Frontal
IC_09 - IPL+AG
IC_10 - MTG
IC_11 - Frontal
IC_12 - SMA
IC_13 - Temporal Pole
IC_14 - Temporal Pole + Fusiform
IC_15 - STG
IC_16 - Middle Occipital?
IC_17 - Cerebellum
IC_18 - Cerebellum
IC_20 - MCC
IC_21 - Temporal Pole + Cerebellum
IC_22 - Insula + Caudate
IC_24 - IPL+Postcentral
IC_26 - Inf+Mid Frontal
IC_28 - Calcarine
IC_29 - MTG
IC_30 - Inf Frontal
`

### **2.1 Source-based Morphometry Loadings' Distributions**

All of the loading features follow a normal distribution. Their distributions and descriptive statistical summary in training and test samples are very similar except `IC_20`. It is an exception because the distribution of `IC_20` in test samples is shifted. This feature may require some preprocessing.

None of the loading features have a visible relationship with any of the targets. Data points of loading features are scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2`, however `age` has a tiny relationship with loading features. This relationship is not easy to detect but it looks like `age` is easier to predict than other target features.

There are some data points which can be classified as mild outliers in all loading features. Models used for predicting targets, should be robust to outliers. The most extreme outlier in loading features belongs to `IC_02`. It is a single data point and it's locations are very close in every target.

In [ ]:
def plot_loading(loading_feature):    
    
    print(f'Loading feature {loading_feature} Statistical Analysis\n{"-" * 42}')
        
    print(f'Mean: {df[loading_feature].mean():.4}  -  Median: {df[loading_feature].median():.4}  -  Std: {df[loading_feature].std():.4}')
    print(f'Min: {df[loading_feature].min():.4}  -  25%: {df[loading_feature].quantile(0.25):.4}  -  50%: {df[loading_feature].quantile(0.5):.4}  -  75%: {df[loading_feature].quantile(0.75):.4}  -  Max: {df[loading_feature].max():.4}')
    print(f'Skew: {df[loading_feature].skew():.4}  -  Kurtosis: {df[loading_feature].kurtosis():.4}')
    missing_values_count = df[df[loading_feature].isnull()].shape[0]
    training_samples_count = df.shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(25, 12), dpi=100, constrained_layout=True)
    title_size = 18
    label_size = 18

    # Loading Feature Training and Test Set Distribution
    sns.distplot(df[df['is_train'] == 1][loading_feature], label='Training', ax=axes[0][0])
    sns.distplot(df[df['is_train'] == 0][loading_feature], label='Test', ax=axes[0][0])
    axes[0][0].set_xlabel('')
    axes[0][0].tick_params(axis='x', labelsize=label_size)
    axes[0][0].tick_params(axis='y', labelsize=label_size)
    axes[0][0].legend()
    axes[0][0].set_title(f'{loading_feature} Distribution in Training and Test Set', size=title_size, pad=title_size)
    
    # Loading Feature vs age
    sns.scatterplot(df[loading_feature], df['age'], ax=axes[0][1])
    axes[0][1].set_title(f'{loading_feature} vs age', size=title_size, pad=title_size)
    axes[0][1].set_xlabel('')
    axes[0][1].set_ylabel('')
    axes[0][1].tick_params(axis='x', labelsize=label_size)
    axes[0][1].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain1_var1
    sns.scatterplot(df[loading_feature], df['domain1_var1'], ax=axes[0][2])
    axes[0][2].set_title(f'{loading_feature} vs domain1_var1', size=title_size, pad=title_size)
    axes[0][2].set_xlabel('')
    axes[0][2].set_ylabel('')
    axes[0][2].tick_params(axis='x', labelsize=label_size)
    axes[0][2].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain1_var2
    sns.scatterplot(df[loading_feature], df['domain1_var2'], ax=axes[1][0])
    axes[1][0].set_title(f'{loading_feature} vs domain1_var2', size=title_size, pad=title_size)
    axes[1][0].set_xlabel('')
    axes[1][0].set_ylabel('')
    axes[1][0].tick_params(axis='x', labelsize=label_size)
    axes[1][0].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain2_var1
    sns.scatterplot(df[loading_feature], df['domain2_var1'], ax=axes[1][1])
    axes[1][1].set_title(f'{loading_feature} vs domain2_var1', size=title_size, pad=title_size)
    axes[1][1].set_xlabel('')
    axes[1][1].set_ylabel('')
    axes[1][1].tick_params(axis='x', labelsize=label_size)
    axes[1][1].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain2_var2
    sns.scatterplot(df[loading_feature], df['domain2_var2'], ax=axes[1][2])
    axes[1][2].set_title(f'{loading_feature} vs domain2_var2', size=title_size, pad=title_size)
    axes[1][2].set_xlabel('')
    axes[1][2].set_ylabel('')
    axes[1][2].tick_params(axis='x', labelsize=label_size)
    axes[1][2].tick_params(axis='y', labelsize=label_size)
    
    plt.show()
    
for loading_feature in sorted(loading_features):
    plot_loading(loading_feature)


### **2.2 Source-based Morphometry Loadings' Correlations**

There are strong correlations between `age` and some loading features that exceed **-0.4**, but none of the loading features are correlated with other targets.

Loading features have decent correlations between themselves that exceed **0.5** and **-0.5**. Some of those high positive correlations belong to feature groups which are from the same part of the brain. However, all features from the same part of the brain are not necessarily correlated with each other, so there is no pattern here.

In [ ]:
loading_target_features = sorted(loading_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(30, 30), dpi=100)

sns.heatmap(df[loading_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=45)
plt.tick_params(axis='y', labelsize=18, rotation=45)

plt.title('Target and Loading Features Correlations', size=25, pad=25)
plt.show()

## **3. Static Functional Network Connectivity**
The second set of features are static functional network connectivity (FNC) matrices. These are the subject-level cross-correlation values among 53 component timecourses estimated from GIG-ICA of resting state functional MRI (fMRI).

There are **1378** features in `fnc.csv` file without `Id`. Those features are named as `Network1(X)_vs_Network2(Y)` and there are **7** different networks. Network names and abbreviations are listed below:

`
SCN - Sub-cortical Network
ADN - Auditory Network
SMN - Sensorimotor Network
VSN - Visual Network
CON - Cognitive-control Network    
DMN - Default-mode Network
CBN - Cerebellar Network
`

Those groups might be useful to analyze **1378** features part by part.

### **3.1. Sub-cortical Network (SCN) Features**

First FNC feature sub-group is SCN features. This is the smallest sub-group of fnc features and there are **10** features in it. This sub-group has cross-correlations with only itself.

SCN features are strongly correlated with each other, but none of the SCN features are correlated with targets. Correlations between target and SCN features are very weak which can be seen at the bottom and right end.

In [ ]:
scn_pattern = r'SCN\(\d+\)_vs_[A-Z]+\(\d+\)'
scn_features = [col for col in fnc_features if re.match(scn_pattern, col)]
scn_target_features = sorted(scn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(25, 25), dpi=100)

sns.heatmap(df[scn_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=75)
plt.tick_params(axis='y', labelsize=18, rotation=0)

plt.title('Target and SCN Features Correlations', size=25, pad=25)
plt.show()

### **3.2. Auditory Network (ADN) Features**

Second FNC feature sub-group is ADN features. This is also the second smallest sub-group of FNC features and there are **11** features in it. This sub-group has cross-correlations with SCN (10) and itself (1).

ADN features are strongly correlated with each other if only ADN is cross-correlated with SCN. There are 10 ADN(2) vs SCN(5) features and they have very strong correlations. There is only one ADN vs ADN (`ADN(56)_vs_ADN(21)`) feature and it doesn't have any significant correlation with anything. Correlations between target and ADN features are very weak which can be seen at the bottom and right end.

In [ ]:
adn_pattern = r'ADN\(\d+\)_vs_[A-Z]+\(\d+\)'
adn_features = [col for col in fnc_features if re.match(adn_pattern, col)]
adn_target_features = sorted(adn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(25, 25), dpi=100)

sns.heatmap(df[adn_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15},
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=75)
plt.tick_params(axis='y', labelsize=18, rotation=0)

plt.title('Target and ADN Features Correlations', size=25, pad=25)
plt.show()

### **3.3. Sensorimotor Network (SMN) Features**

Third FNC feature sub-group is SMN features. This is a large sub-group compared to previous ones and there are **99** features in it. This sub-group has cross-correlations with SCN (45), ADN (18) and itself (36).

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect.

Every cross-correlation of SMN have very strong correlations inside the second network groups. This explains the bright red blocks near the diagonal axis.

* `SMN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `SMN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `SMN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`

Every cross-correlation of SMN have moderate correlations inside the first network groups. This explains the repeating orange and blue color blocks along the vertical and horizontal axis.

* `SMN(X)_vs_ADN(Y)`: `Y` is moderately correlated with every different `X` value for `ADN` 
* `SMN(X)_vs_SCN(Y)`: `Y` is moderately correlated with every different `X` value for `SCN` 
* `SMN(X)_vs_SMN(Y)`: `Y` is moderately correlated with every different `X` value for `SMN`

Correlations between target and SMN features are very weak which can be seen at the bottom and right end.

In [ ]:
smn_pattern = r'SMN\(\d+\)_vs_[A-Z]+\(\d+\)'
smn_features = [col for col in fnc_features if re.match(smn_pattern, col)]
smn_target_features = sorted(smn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(40, 40), dpi=100)

sns.heatmap(df[smn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and SMN Features Correlations', size=50, pad=50)
plt.show()

### **3.4. Visual Network (VSN) Features** 

Fourth FNC feature sub-group is VSN features. This is a very large sub-group compared to previous ones and there are **180** features in it. This sub-group has cross-correlations with SCN (45), ADN (18), SMN (81) and itself (36).

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect. The same pattern from SMN features can be seen on VSN features as well.

Every cross-correlation of VSN have very strong correlations inside the second network groups. This explains the bright red blocks near the diagonal axis.

* `VSN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `VSN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `VSN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`
* `VSN(X)_vs_VSN(Y)`: `X` is strongly correlated with every different `Y` value for `VSN`

Every cross-correlation of VSN have moderate correlations inside the first network groups. This explains the repeating orange and blue color blocks along the vertical and horizontal axis.

* `VSN(X)_vs_ADN(Y)`: `Y` is moderately correlated with every different `X` value for `ADN` 
* `VSN(X)_vs_SCN(Y)`: `Y` is moderately correlated with every different `X` value for `SCN` 
* `VSN(X)_vs_SMN(Y)`: `Y` is moderately correlated with every different `X` value for `SMN`
* `VSN(X)_vs_VSN(Y)`: `Y` is moderately correlated with every different `X` value for `VSN`

Correlations between target and VSN features are very weak which can be seen at the bottom and right end.

In [ ]:
vsn_pattern = r'VSN\(\d+\)_vs_[A-Z]+\(\d+\)'
vsn_features = [col for col in fnc_features if re.match(vsn_pattern, col)]
vsn_target_features = sorted(vsn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(40, 40), dpi=100)

sns.heatmap(df[vsn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and VSN Features Correlations', size=50, pad=50)
plt.show()

### **3.5. Cognitive-control Network (CON) Features** 

Fifth FNC feature sub-group is CON features. This is the largest sub-group and there are **561** features in it. This sub-group has cross-correlations with SCN (85), ADN (34), SMN (153), VSN (153) and itself (136).

It is even hard to identify correlations at block level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect. The same pattern from SMN and VSN features can be seen on CON features as well.

Every cross-correlation of CON have very strong correlations inside the second network groups. This explains the bright red blocks near the diagonal axis.

* `CON(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `CON(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `CON(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`
* `CON(X)_vs_VSN(Y)`: `X` is strongly correlated with every different `Y` value for `VSN`
* `CON(X)_vs_CON(Y)`: `X` is strongly correlated with every different `Y` value for `CON`

Every cross-correlation of CON have weak correlations inside the first network groups. This explains the light orange and light blue color blocks everywhere except near of diagonal axis.

Correlations between target and CON features are very weak which can be seen at the bottom and right end.

In [ ]:
con_pattern = r'CON\(\d+\)_vs_[A-Z]+\(\d+\)'
con_features = [col for col in fnc_features if re.match(con_pattern, col)]
con_target_features = sorted(con_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(50, 50), dpi=100)

sns.heatmap(df[con_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and CON Features Correlations', size=50, pad=50)
plt.show()

### **3.6. Default-mode Network (DMN) Features** 

Sixth FNC feature sub-group is DMN features. This is the second largest sub-group and there are **315** features in it. This sub-group has cross-correlations with SCN (35), ADN (14), SMN (63), VSN (63), CON (119) and itself (21).

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect. The same pattern from SMN, VSN and CON features can be seen on DMN features as well.

Every cross-correlation of DMN have very strong correlations inside the second network groups. This explains the bright red and blue blocks near the diagonal axis.

* `DMN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `DMN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `DMN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`
* `DMN(X)_vs_VSN(Y)`: `X` is strongly correlated with every different `Y` value for `VSN`
* `DMN(X)_vs_CON(Y)`: `X` is strongly correlated with every different `Y` value for `CON`
* `DMN(X)_vs_DMN(Y)`: `X` is strongly correlated with every different `Y` value for `DMN`

Every cross-correlation of DMN have weak correlations inside the first network groups. This explains the light orange and light blue color blocks everywhere except near of diagonal axis. There are also repeating light orange and light blue diagonal lines. 

Correlations between target and DMN features are very weak which can be seen at the bottom and right end.

In [ ]:
dmn_pattern = r'DMN\(\d+\)_vs_[A-Z]+\(\d+\)'
dmn_features = [col for col in fnc_features if re.match(dmn_pattern, col)]
dmn_target_features = sorted(dmn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(50, 50), dpi=100)

sns.heatmap(df[dmn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and DMN Features Correlations', size=50, pad=50)
plt.show()

### **3.7. Cerebellar Network (CBN) Features** 

Seventh FNC feature sub-group is CBN features. This is a large sub-group and there are **202** features in it. This sub-group has cross-correlations with SCN (20), ADN (8), SMN (36), VSN (36), CON (68), DMN (28) and itself (6).

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect. The same pattern from SMN, VSN, CON and DMN features can be seen on CBN features as well.

Every cross-correlation of CBN have very strong correlations inside the second network groups. This explains the bright red and blue blocks near the diagonal axis.

* `CBN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `CBN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `CBN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`
* `CBN(X)_vs_VSN(Y)`: `X` is strongly correlated with every different `Y` value for `VSN`
* `CBN(X)_vs_CON(Y)`: `X` is strongly correlated with every different `Y` value for `CON`
* `CBN(X)_vs_DMN(Y)`: `X` is strongly correlated with every different `Y` value for `DMN`
* `CBN(X)_vs_CBN(Y)`: `X` is strongly correlated with every different `Y` value for `CBN`

Every cross-correlation of CBN have moderate correlations inside the first network groups. This explains the repeating orange and blue color blocks along the vertical and horizontal axis, and repeating bright red and orange diagonal lines.

* `CBN(X)_vs_ADN(Y)`: `Y` is strongly correlated with every different `X` value for `ADN` 
* `CBN(X)_vs_SCN(Y)`: `Y` is strongly correlated with every different `X` value for `SCN` 
* `CBN(X)_vs_SMN(Y)`: `Y` is strongly correlated with every different `X` value for `SMN`
* `CBN(X)_vs_VSN(Y)`: `Y` is strongly correlated with every different `X` value for `VSN`
* `CBN(X)_vs_CON(Y)`: `X` is strongly correlated with every different `Y` value for `CON`
* `CBN(X)_vs_DMN(Y)`: `X` is strongly correlated with every different `Y` value for `DMN`
* `CBN(X)_vs_CBN(Y)`: `X` is strongly correlated with every different `Y` value for `CBN`

Correlations between target and CBN features are very weak which can be seen at the bottom and right end.

In [ ]:
cbn_pattern = r'CBN\(\d+\)_vs_[A-Z]+\(\d+\)'
cbn_features = [col for col in fnc_features if re.match(cbn_pattern, col)]
cbn_target_features = sorted(cbn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(50, 50), dpi=100)

sns.heatmap(df[cbn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and CBN Features Correlations', size=50, pad=50)
plt.show()

### **3.8. Inference**
FNC features from the same group are more likely to be correlated with each other. Correlations from different groups doesn't seem to be very strong. Even in the same groups, there are lots of weak correlations. FNC features from the same groups must be in the same network in order to have strong correlation. FNC features in the same network don't have strong correlations if they are not in the same group.

The entire heatmap of FNC features is rendered below but it is very hard to derive any meaning from it.

In [ ]:
fig = plt.figure(figsize=(60, 60), dpi=100)

sns.heatmap(df[fnc_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.title('Target and All FNC Features Correlations', size=60, pad=60)
plt.show()

## **4. Sites**

`reveal_ID_site2.csv` a list of subject IDs whose data was collected with a different scanner than the train samples.

> Models are expected to generalize on data from a different scanner/site (site 2). All subjects from site 2 were assigned to the test set, so their scores are not available. While there are fewer site 2 subjects than site 1 subjects in the test set, the total number of subjects from site 2 will not be revealed until after the end of the competition. To make it more interesting, the IDs of some site 2 subjects have been revealed below. Use this to inform your models about site effects. Site effects are a form of bias. To generalize well, models should learn features that are not related to or driven by site effects.

This paragraph is taken from the data tab. It means data is collected from two different sources, site 1 and 2. This is the reason of train and test set feature distribution discrepancies.

* All of the data in training set is collected from site 1.
* All of the data collected from site 2 are in test set but the entire test set is not collected from site 2.

In this case, there is no way to find out site 1 samples in test set perfectly, but they can be predicted with a classifier by looking at feature distributions.

In [ ]:
site2_ids = pd.read_csv('../input/trends-assessment-prediction/reveal_ID_site2.csv').values.flatten()

df['site'] = 0
df.loc[df['is_train'] == 1, 'site'] = 1
df.loc[df['Id'].isin(site2_ids), 'site'] = 2
df['site'] = df['site'].astype(np.uint8)

del site2_ids

In [ ]:
fig = plt.figure(figsize=(10, 6), dpi=100)
sns.barplot(x=df['site'].value_counts().index, y=df['site'].value_counts())

percentages = [(count / df['site'].value_counts().sum() * 100).round(2) for count in df['site'].value_counts()]
plt.ylabel('')
plt.xticks(np.arange(3), [f'No Site (%{percentages[1]})', f'Site 1 (%{percentages[0]})', f'Site 2 (%{percentages[2]})'])
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
plt.title('Site Counts', size=15, pad=15)

plt.show()

### **4.1. Loading Features Site Distributions**

It is hard to make an accurate analysis on feature distributions for different sites because sample sizes are different for site 1 and 2. There are **5877** site 1 samples (training set) and **510** verified site 2 samples. That's why train/test distribution of the feature should be plotted as well in order to measure the shift in distribution.

There are three types of shifts in the distributions ploted below:

* Mild shift in site 1/2 feature distribution but it can't be seen on train/test feature distribution (Mean shift close to ±0.001). Those shifts can be observed due to small sample size of site 2 subjects, but they are suspicious.
* Severe shift in site 1/2 feature distribution that it can also be seen on train/test feature distribution (Mean shift greater than ±0.002). Those features should be eliminated.
* No shift in both site 1/2 and train/test feature distributions. Those features can be trusted. 

In [ ]:
def plot_site_distribution(feature):    
    
    print(f'{feature} Site Distribution Analysis\n{"-" * 32}')
        
    print(f'Site 1 Mean: {df[df["site"] == 1][feature].mean():.4}  -  Median: {df[df["site"] == 1][feature].median():.4}  -  Std: {df[df["site"] == 1][feature].std():.4}')
    print(f'Site 2 Mean: {df[df["site"] == 2][feature].mean():.4}  -  Median: {df[df["site"] == 2][feature].median():.4}  -  Std: {df[df["site"] == 2][feature].std():.4}')
    print(f'\nSite 1 Min: {df[df["site"] == 1][feature].min():.4}  -  25%: {df[df["site"] == 1][feature].quantile(0.25):.4}  -  50%: {df[df["site"] == 1][feature].quantile(0.5):.4}  -  75%: {df[df["site"] == 1][feature].quantile(0.75):.4}  -  Max: {df[df["site"] == 1][feature].max():.4}')
    print(f'Site 2 Min: {df[df["site"] == 2][feature].min():.4}  -  25%: {df[df["site"] == 2][feature].quantile(0.25):.4}  -  50%: {df[df["site"] == 2][feature].quantile(0.5):.4}  -  75%: {df[df["site"] == 2][feature].quantile(0.75):.4}  -  Max: {df[df["site"] == 2][feature].max():.4}')

    fig, axes = plt.subplots(ncols=2, figsize=(20, 6), dpi=100)

    sns.distplot(df[df['site'] == 1][feature], label='Site 1', ax=axes[0])
    sns.distplot(df[df['site'] == 2][feature], label='Site 2', ax=axes[0])
    sns.distplot(df[df['is_train'] == 1][feature], label='Training', ax=axes[1])
    sns.distplot(df[df['is_train'] == 0][feature], label='Test', ax=axes[1])
    
    for i in range(2):
        axes[i].set_xlabel('')
        axes[i].tick_params(axis='x', labelsize=15)
        axes[i].tick_params(axis='y', labelsize=15)
        axes[i].legend()
    axes[0].set_title(f'{feature} Distribution in Site 1 and 2', size=18, pad=18)
    axes[1].set_title(f'{feature} Distribution in Training and Test Set', size=18, pad=18)
    
    plt.show()
    
for loading_feature in sorted(loading_features):
    plot_site_distribution(loading_feature)

### **4.2. FNC Features Site Distributions**

There are 1378 FNC features so they can't be analyzed one by one. Kolmogorov–Smirnov test calculated on site 1 and site 2 feature distribution can be used for feature selection. If the KS statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same. I used **0.125** for KS statistic threshold and selected features that are exceeding the threshold.

In [ ]:
def get_distribution_difference(feature):
    site1_values = df[df['site'] == 1][feature].values
    site2_values = df[df['site'] == 2][feature].values
    return feature, ks_2samp(site1_values, site2_values)

ks_threshold = 0.125
shifted_features = [fnc_feature for fnc_feature in fnc_features if get_distribution_difference(fnc_feature)[1][0] > ks_threshold]        

The same things in loading features can be observed in FNC features as well. Slightly shifted train/test feature distributions are shifted even more in site 1/2. However, none of the feature shifts are as severe as `IC_20` in loading features.

**0.125** is an arbitrary number and distribution difference can be measured with other ways as well, but feature selection should definitely be based on site 1 and 2 feature distribution difference.

In [ ]:
for feature in sorted(shifted_features):
    plot_site_distribution(feature)

### **4.3. Site Classification**

Even though it is impossible to predict the sites of unlabeled samples perfectly, shifted features can be used in a classifier and predict the sites to some degree. Loading features should be used in this classifier along with shifted FNC features.

In [ ]:
site_predictors = shifted_features + loading_features

X_train = df.loc[df['site'] > 0, site_predictors]
y_train = df.loc[df['site'] > 0, 'site']
X_test = df.loc[df['site'] == 0, site_predictors]

Expected values closer to **1** and **2** in `site_predicted`, could possible be added to the original `site` feature since they are confident predictions.

`IC_20`, `IC_18`, `IC_11`, `IC_21` and `IC_05` are at the top of feature importance in site classifier model. It looks like FNC features don't contribute site classifier model as much as loading features. `IC_20` is the most dangerous feature without any doubt but other loading features should be selected very carefully in main models.

In [ ]:
df['site_predicted'] = 0

K = 2
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)

oof_scores = []
feature_importance = pd.DataFrame(np.zeros((X_train.shape[1], K)), columns=[f'Fold_{i}_Importance' for i in range(1, K + 1)], index=X_train.columns)

site_model_parameters = {
    'num_iterations': 500,
    'early_stopping_round': 50,
    'num_leaves': 2 ** 5, 
    'learning_rate': 0.05,
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'feature_fraction': 0.9,
    'feature_fraction_bynode': 0.9,
    'lambda_l1': 0,
    'lambda_l2': 0,
    'max_depth': -1,
    'objective': 'regression',
    'seed': SEED,
    'feature_fraction_seed': SEED,
    'bagging_seed': SEED,
    'drop_seed': SEED,
    'data_random_seed': SEED,
    'boosting_type': 'gbdt',
    'verbose': 1,
    'metric': 'rmse',
    'n_jobs': -1,   
}

print('Running LightGBM Site Classifier Model\n' + ('-' * 38) + '\n')

for fold, (trn_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):

    trn_data = lgb.Dataset(X_train.iloc[trn_idx, :], label=y_train.iloc[trn_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx, :], label=y_train.iloc[val_idx])                
    site_model = lgb.train(site_model_parameters, trn_data, valid_sets=[trn_data, val_data], verbose_eval=50)
    feature_importance.iloc[:, fold - 1] = site_model.feature_importance(importance_type='gain')

    site_oof_predictions = site_model.predict(X_train.iloc[val_idx, :], num_iteration=site_model.best_iteration)
    df.loc[X_train.iloc[val_idx, :].index, 'site_predicted'] = site_oof_predictions
    
    site_test_predictions = site_model.predict(X_test, num_iteration=site_model.best_iteration)
    df.loc[X_test.index, 'site_predicted'] += site_test_predictions / K

    oof_score = f1_score(y_train.iloc[val_idx], np.clip(np.round(site_oof_predictions), 1, 2))
    oof_scores.append(oof_score)            
    print(f'\nFold {fold} - F1 Score {oof_score:.6}\n')
    
df['site_predicted'] = df['site_predicted'].astype(np.float32)
oof_f1_score = f1_score(df.loc[df['site'] > 0, 'site'], np.clip(np.round(df.loc[df['site'] > 0, 'site_predicted']), 1, 2))

print('-' * 38)
print(f'LightGBM Site Classifier Model Mean F1 Score {np.mean(oof_scores):.6} [STD:{np.std(oof_scores):.6}]')
print(f'LightGBM Site Classifier Model OOF F1 Score {oof_f1_score:.6}')

plt.figure(figsize=(20, 20))
feature_importance['Mean_Importance'] = feature_importance.sum(axis=1) / K
feature_importance.sort_values(by='Mean_Importance', inplace=True, ascending=False)
sns.barplot(x='Mean_Importance', y=feature_importance.index, data=feature_importance)

plt.xlabel('')
plt.tick_params(axis='x', labelsize=18)
plt.tick_params(axis='y', labelsize=18)
plt.title('LightGBM Site Classifier Model Feature Importance (Gain)', size=20, pad=20)

plt.show()

### **4.4. Private Leaderboard Simulation**
If the expected values predicted by the site classifier are rounded, there are **32** samples classified as site 2. This operation ceils the values greater than **.5** and floors the values lesser than **.5**. It yields very few samples for models to use as a validation set.

If this threshold is set to **.2**, a 0.9 train/test split can be achieved with samples which slightly look like they are from site 2. I tried to simulate the private leaderboard by:

* Training on **5342** samples which are classified as site 1 (`site_predicted` expected value < 1.2)
* Validate on **535** samples which slightly look like site 2 (`site_predicted` expected value >= 1.2)

In [ ]:
print('Site Classifier expected values are rounded with .5 threshold\n')
print(df.loc[df['is_train'] == 1, 'site_predicted'].round(0).astype(np.uint8).value_counts())

print('\nSite Classifier expected values are rounded with .2 threshold\n')
df.loc[df['site_predicted'] > 1.2, 'site_predicted'] = 2
df['site_predicted'] = df['site_predicted'].round().astype(np.uint8)
print(df.loc[df['is_train'] == 1, 'site_predicted'].round(0).astype(np.uint8).value_counts())

Ridge regression is used in this simulation and the results are listed below. The gap between training and validation NAE looks very realistic based on the train/test discrepancies, especially for `age`.

`
age Train NAE 0.129934 - Validation NAE 0.154733
domain1_var1 Train NAE 0.143931 - Validation NAE 0.147178
domain1_var2 Train NAE 0.150529 - Validation NAE 0.146966
domain2_var1 Train NAE 0.176281 - Validation NAE 0.189226
domain2_var2 Train NAE 0.171305 - Validation NAE 0.171517
`

Even though Ridge regression scores **0.159** on public leaderboard, it scored **0.161** on slightly different samples. This means the private leaderboard score will be even worse.

`Train Weighted NAE 0.151338 - Validation Weighted NAE 0.161025`

## **5. Component Spatial Maps**

The third set of features are the component spatial maps (SM). These are the subject-level 3D images of 53 spatial networks estimated from GIG-ICA of resting state functional MRI (fMRI). Those features are contained under the directories below:

* `fMRI_train` - a folder containing 53 3D spatial maps for train samples in .mat format
* `fMRI_test` - a folder containing 53 3D spatial maps for test samples in .mat format

Those explanations are written under the Data tab on the competition page, but they are not clear. `fMRI_train` and `fMRI_test` are directories in which there are **5877** samples. Each sample is a `.mat` file that contains **53** 3D spatial maps.

This means there are **311481** 3D spatial maps in both `fMRI_train` and `fMRI_test`, and this is the reason why competition data is **166.59 GB**.

In [ ]:
print(f'Train fMRI Samples in fMRI_train: {len(os.listdir("../input/trends-assessment-prediction/fMRI_train"))}')
print(f'Test fMRI Samples in fMRI_test: {len(os.listdir("../input/trends-assessment-prediction/fMRI_test"))}')

### **5.1 MATLAB Files**

The `.mat` files in this competition can be read in Python using `h5py.File`. HDF5 files work generally like standard Python file objects. They support standard modes like r/w/a, and should be closed when they are no longer in use. For an example, the first file (`10001.mat`) inside the `fMRI_train` is read in the cell below.

In [ ]:
matlab_file = h5py.File(f'../input/trends-assessment-prediction/fMRI_train/10001.mat', 'r')
matlab_file

The `.mat` files are using dictionary interface so data inside the file can be accessed with `keys()` and `values()` methods. `.keys()` method shows us there is only one key in the file and it is `SM_feature`. That is where the data is stored.

Accessing the data using `matlab_file["SM_feature"]` shows us, each `.mat` file is a 4D HDF5 dataset with the shape of `(53, 52, 63, 53)`. Those are the **53** 3D spatial maps.

In [ ]:
print(f'.keys() -> {matlab_file.keys()}')
print(f'.values() -> {matlab_file.values()}')

print('\nmatlab_file["SM_feature"] ->', matlab_file['SM_feature'])

### **5.2 Visualizing 3D Data on 2D Space**

Those **53** 3D spatial maps can be visualized with `nilearn` package. However, 3D spatial maps have to be drawn on a reference image and the reference image is `fMRI_mask.nii` file that is shared in the competition data. Niimg-like objects (`.nii` files) can be load from filenames or list of filenames with `nilearn` as well.

Before the 3D spatial maps are loaded, it's necessary to reorient the axis, since h5py flips axis order. The axis of data is reoriented in reverse order.

Finally, 3D spatial maps can be loaded as a Niimg-like object with `nilearn.image.new_img_like` by using the `fMRI_mask.nii` as a reference image.

In [ ]:
# Loading reference image
fmri_mask = nl.image.load_img('../input/trends-assessment-prediction/fMRI_mask.nii')

# Reorienting the axis of 3D spatial map
spatial_maps = np.moveaxis(matlab_file['SM_feature'][()], [0, 1, 2, 3], [3, 2, 1, 0]) 

# Loading 3D spatial maps
spatial_maps_niimg = nl.image.new_img_like(ref_niimg=fmri_mask,
                                           data=spatial_maps,
                                           affine=fmri_mask.affine,
                                           copy_header=True)

After the 3D spatial maps are loaded as Niimg-like objects, they can be iterated with `nilearn.image.iter_img` and visualized one by one with `nilearn.plotting.plot_stat_map`. The **53** 3D spatial maps that are visualized below, belong to a single sample, `10001.mat`. There are **5877** samples in both `fMRI_train` and `fMRI_test`.

`nilearn` is very useful to visualize 3D spatial maps because `plot_stat_map` plots the data from three different angles which makes it easier to see the correlated parts of brain.

In [ ]:
fig, axes = plt.subplots(nrows=53, figsize=(30, 300))

for i, img in enumerate(list(nl.image.iter_img(spatial_maps_niimg))):
    nlplt.plot_stat_map(stat_map_img=img,
                        bg_img='ch2better.nii',
                        title=f'10001.mat Spatial Map {i + 1} plot_stat_map',
                        axes=axes[i],
                        threshold=1,
                        display_mode='ortho',
                        annotate=False,
                        draw_cross=True,
                        colorbar=False)

There are various ways to visualize 3D spatial maps other than `plot_stat_map`. 5 different plotting functions used in the visualizations below in every row. Those plotting functions are `plot_glass_brain`, `plot_roi`, `plot_anat`, `plot_epi` and `plot_img`. Some of those visualizations are more detailed and some of them are faster.

`threshold` parameter is used for reducing the noise in visualizations. If `None` is given, the image is not thresholded. If a number is given, it is used to threshold the image: values below the threshold (in absolute value) are plotted as transparent.


In [ ]:
img = list(nl.image.iter_img(spatial_maps_niimg))[0]

fig, axes = plt.subplots(nrows=5, figsize=(30, 50))

nlplt.plot_glass_brain(img,
                       title='10001.mat Spatial Map 1 plot_glass_brain',
                       threshold=3,
                       black_bg=True,
                       axes=axes[0])
nlplt.plot_roi(img,
               title='10001.mat Spatial Map 1 plot_roi',
               threshold=3,
               black_bg=True,
               axes=axes[1])

nlplt.plot_anat(img,
                title='10001.mat Spatial Map 1 plot_anat',
                axes=axes[2])

nlplt.plot_epi(img,
               title='10001.mat Spatial Map 1 plot_epi',
               axes=axes[3])

nlplt.plot_img(img,
               title='10001.mat Spatial Map 1 plot_img',
               axes=axes[4])

plt.show()

### **5.3 Visualizing 3D Data on 3D Space**
3D spatial maps can also be visualized on 3D space with `nilearn` package. `view_img_on_surf` used as the plotting function and it renders the 3D data onto a 3D brain figure. 3D visualizations can be thresholded with `threshold` parameter as well. 

In [ ]:
view = nlplt.view_img_on_surf(img,
                              title='10001.mat Spatial Map 1 view_img_on_surf',
                              title_fontsize=20,
                              threshold=1,
                              black_bg=False)
view.open_in_browser()
view

Columns in `fnc.csv`, `loading.csv` and `train_scores.csv` are merged on `Id`. Two features `is_train` and `site` are created for labeling different datasets and sites. Data is saved in pickle format for faster save and load time.

In [ ]:
df.to_pickle('trends_tabular_data.pkl')

print(f'TReNDS Tabular Data Shape = {df.shape}')
print(f'TReNDS Tabular Data Memory Usage = {df.memory_usage().sum() / 1024 ** 2:.2f} MB')

# visualization using nilearn

In [ ]:
# !wget https://github.com/Chaogan-Yan/DPABI/raw/master/Templates/ch2better.nii

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
mask_filename = '../input/trends-assessment-prediction/fMRI_mask.nii'
subject_filename = '../input/trends-assessment-prediction/fMRI_train/10004.mat'
smri_filename = 'ch2better.nii'
mask_niimg = nl.image.load_img(mask_filename)

def load_subject(filename, mask_niimg):
    """
    Load a subject saved in .mat format with
        the version 7.3 flag. Return the subject
        niimg, using a mask niimg as a template
        for nifti headers.
        
    Args:
        filename    <str>            the .mat filename for the subject data
        mask_niimg  niimg object     the mask niimg object used for nifti headers
    """
    subject_data = None
    with h5py.File(subject_filename, 'r') as f:
        subject_data = f['SM_feature'][()]
    # It's necessary to reorient the axes, since h5py flips axis order
    subject_data = np.moveaxis(subject_data, [0,1,2,3], [3,2,1,0])
    subject_niimg = nl.image.new_img_like(mask_niimg, subject_data, affine=mask_niimg.affine, copy_header=True)
    return subject_niimg
subject_niimg = load_subject(subject_filename, mask_niimg)
print("Image shape is %s" % (str(subject_niimg.shape)))
num_components = subject_niimg.shape[-1]
print("Detected {num_components} spatial maps".format(num_components=num_components))

In [ ]:
nlplt.plot_prob_atlas(subject_niimg, bg_img=smri_filename, view_type='filled_contours', draw_cross=False, title='All %d spatial maps' % num_components, threshold='auto')

In [ ]:
grid_size = int(np.ceil(np.sqrt(num_components)))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size*10, grid_size*10))
[axi.set_axis_off() for axi in axes.ravel()]
row = -1
for i, cur_img in enumerate(nl.image.iter_img(subject_niimg)):
    col = i % grid_size
    if col == 0:
        row += 1
    nlplt.plot_stat_map(cur_img, bg_img=smri_filename, title="IC %d" % i, axes=axes[row, col], threshold=3, colorbar=False)

In [ ]:
# import matplotlib.pyplot as plt

from nilearn import datasets
haxby_dataset = datasets.fetch_haxby()   # load dataset

# print basic information on the dataset
print('First subject anatomical nifti image (3D) is at: %s' %
      haxby_dataset.anat[0])
print('First subject functional nifti image (4D) is at: %s' %
      haxby_dataset.func[0])  # 4D data

# Build the mean image because we have no anatomic data
from nilearn import image
func_filename = haxby_dataset.func[0]
mean_img = image.mean_img(func_filename)

z_slice = -14

fig = plt.figure(figsize=(4, 5.4), facecolor='k')

from nilearn.plotting import plot_anat, show
display = plot_anat(mean_img, display_mode='z', cut_coords=[z_slice],
                    figure=fig)
mask_vt_filename = haxby_dataset.mask_vt[0]
mask_house_filename = haxby_dataset.mask_house[0]
mask_face_filename = haxby_dataset.mask_face[0]
display.add_contours(mask_vt_filename, contours=1, antialiased=False,
                     linewidths=4., levels=[0], colors=['red'])
display.add_contours(mask_house_filename, contours=1, antialiased=False,
                     linewidths=4., levels=[0], colors=['blue'])
display.add_contours(mask_face_filename, contours=1, antialiased=False,
                     linewidths=4., levels=[0], colors=['limegreen'])

# We generate a legend using the trick described on
# http://matplotlib.sourceforge.net/users/legend_guide.httpml#using-proxy-artist
from matplotlib.patches import Rectangle
p_v = Rectangle((0, 0), 1, 1, fc="red")
p_h = Rectangle((0, 0), 1, 1, fc="blue")
p_f = Rectangle((0, 0), 1, 1, fc="limegreen")
plt.legend([p_v, p_h, p_f], ["vt", "house", "face"])

show()

In [ ]:
# from nilearn import datasets

# haxby dataset to have EPI images and masks
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('First subject anatomical nifti image (3D) is at: %s' %
      haxby_dataset.anat[0])
print('First subject functional nifti image (4D) is at: %s' %
      haxby_dataset.func[0])  # 4D data

haxby_anat_filename = haxby_dataset.anat[0]
haxby_mask_filename = haxby_dataset.mask_vt[0]
haxby_func_filename = haxby_dataset.func[0]

# one motor contrast map from NeuroVault
motor_images = datasets.fetch_neurovault_motor_task()
stat_img = motor_images.images[0]

In [ ]:
from nilearn import plotting

# Visualizing t-map image on EPI template with manual
# positioning of coordinates using cut_coords given as a list
plotting.plot_stat_map(stat_img,
                       threshold=3, title="plot_stat_map",
                       cut_coords=[36, -27, 66])

In [ ]:
view = plotting.view_img(stat_img, threshold=3)
# In a Jupyter notebook, if ``view`` is the output of a cell, it will
# be displayed below the cell
view

In [ ]:
plotting.plot_glass_brain(stat_img, title='plot_glass_brain',
                          threshold=3)

In [ ]:
plotting.plot_anat(haxby_anat_filename, title="plot_anat")

In [ ]:
plotting.plot_roi(haxby_mask_filename, bg_img=haxby_anat_filename,
                  title="plot_roi")

In [ ]:
# Import image processing tool
# from nilearn import image

# Compute the voxel_wise mean of functional images across time.
# Basically reducing the functional image from 4D to 3D
mean_haxby_img = image.mean_img(haxby_func_filename)

# Visualizing mean image (3D)
plotting.plot_epi(mean_haxby_img, title="plot_epi")

In [ ]:
# haxby dataset to have anatomical image, EPI images and masks
#haxby_dataset = datasets.fetch_haxby()
haxby_anat_filename = haxby_dataset.anat[0]
haxby_mask_filename = haxby_dataset.mask_vt[0]
haxby_func_filename = haxby_dataset.func[0]

# localizer dataset to have contrast maps
motor_images = datasets.fetch_neurovault_motor_task()
stat_img = motor_images.images[0]

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='ortho',
                       cut_coords=[36, -27, 60],
                       title="display_mode='ortho', cut_coords=[36, -27, 60]")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='z', cut_coords=5,
                       title="display_mode='z', cut_coords=5")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='x',
                       cut_coords=[-36, 36],
                       title="display_mode='x', cut_coords=[-36, 36]")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='y', cut_coords=1,
                       title="display_mode='y', cut_coords=1")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='z',
                       cut_coords=1, colorbar=False,
                       title="display_mode='z', cut_coords=1, colorbar=False")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='xz',
                       cut_coords=[36, 60],
                       title="display_mode='xz', cut_coords=[36, 60]")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='yx',
                       cut_coords=[-27, 36],
                       title="display_mode='yx', cut_coords=[-27, 36]")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='yz',
                       cut_coords=[-27, 60],
                       title="display_mode='yz', cut_coords=[-27, 60]")

In [ ]:
plotting.plot_stat_map(stat_img, display_mode='tiled',
                       cut_coords=[36, -27, 60],
                       title="display_mode='tiled'")

In [ ]:
# from nilearn import image

# Compute voxel-wise mean functional image across time dimension. Now we have
# functional image in 3D assigned in mean_haxby_img
mean_haxby_img = image.mean_img(haxby_func_filename)

In [ ]:
display = plotting.plot_anat(mean_haxby_img, title="add_edges")

# We are now able to use add_edges method inherited in plotting object named as
# display. First argument - anatomical image  and by default edges will be
# displayed as red 'r', to choose different colors green 'g' and  blue 'b'.
display.add_edges(haxby_anat_filename)

In [ ]:
# As seen before, we call the `plot_anat` function with a background image
# as first argument, in this case again the mean fMRI image and argument
# `cut_coords` as list for manual cut with coordinates pointing at masked
# brain regions
display = plotting.plot_anat(mean_haxby_img, title="add_contours",
                             cut_coords=[-34, -39, -9])
# Now use `add_contours` in display object with the path to a mask image from
# the Haxby dataset as first argument and argument `levels` given as list
# of values to select particular level in the contour to display and argument
# `colors` specified as red 'r' to see edges as red in color.
# See help on matplotlib.pyplot.contour to use more options with this method
display.add_contours(haxby_mask_filename, levels=[0.5], colors='r')

In [ ]:
display = plotting.plot_anat(mean_haxby_img,
                             title="add_contours with filled=True",
                             cut_coords=[-34, -39, -9])

# By default, no color fillings will be shown using `add_contours`. To see
# contours with color fillings use argument filled=True. contour colors are
# changed to blue 'b' with alpha=0.7 sets the transparency of color fillings.
# See help on matplotlib.pyplot.contourf to use more options given that filled
# should be True
display.add_contours(haxby_mask_filename, filled=True, alpha=0.7,
                     levels=[0.5], colors='b')

In [ ]:
display = plotting.plot_anat(mean_haxby_img, title="add_markers",
                             cut_coords=[-34, -39, -9])

# Coordinates of seed regions should be specified in first argument and second
# argument `marker_color` denotes color of the sphere in this case yellow 'y'
# and third argument `marker_size` denotes size of the sphere
coords = [(-34, -39, -9)]
display.add_markers(coords, marker_color='y', marker_size=100)

In [ ]:
display = plotting.plot_anat(mean_haxby_img,
                             title="adding a scale bar",
                             cut_coords=[-34, -39, -9])
display.annotate(scalebar=True)

In [ ]:
display = plotting.plot_anat(mean_haxby_img,
                             title="adding a scale bar",
                             cut_coords=[-34, -39, -9])
display.annotate(scalebar=True, scale_size=25, scale_units='mm')

In [ ]:
# from nilearn import datasets

rest_dataset = datasets.fetch_development_fmri(n_subjects=20)
func_filenames = rest_dataset.func
confounds = rest_dataset.confounds

In [ ]:
# Import dictionary learning algorithm from decomposition module and call the
# object and fit the model to the functional datasets
from nilearn.decomposition import DictLearning

# Initialize DictLearning object
dict_learn = DictLearning(n_components=8, smoothing_fwhm=6.,
                          memory="nilearn_cache", memory_level=2,
                          random_state=0)
# Fit to the data
dict_learn.fit(func_filenames)
# Resting state networks/maps in attribute `components_img_`
# Note that this attribute is implemented from version 0.4.1.
# For older versions, see the note section above for details.
components_img = dict_learn.components_img_

# Visualization of functional networks
# Show networks using plotting utilities
# from nilearn import plotting

plotting.plot_prob_atlas(components_img, view_type='filled_contours',
                         title='Dictionary Learning maps')

In [ ]:
# Import Region Extractor algorithm from regions module
# threshold=0.5 indicates that we keep nominal of amount nonzero voxels across all
# maps, less the threshold means that more intense non-voxels will be survived.
from nilearn.regions import RegionExtractor

extractor = RegionExtractor(components_img, threshold=0.5,
                            thresholding_strategy='ratio_n_voxels',
                            extractor='local_regions',
                            standardize=True, min_region_size=1350)
# Just call fit() to process for regions extraction
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

# Visualization of region extraction results
title = ('%d regions are extracted from %d components.'
         '\nEach separate color of region indicates extracted region'
         % (n_regions_extracted, 8))
plotting.plot_prob_atlas(regions_extracted_img, view_type='filled_contours',
                         title=title)

In [ ]:
# First we need to do subjects timeseries signals extraction and then estimating
# correlation matrices on those signals.
# To extract timeseries signals, we call transform() from RegionExtractor object
# onto each subject functional data stored in func_filenames.
# To estimate correlation matrices we import connectome utilities from nilearn
from nilearn.connectome import ConnectivityMeasure

correlations = []
# Initializing ConnectivityMeasure object with kind='correlation'
connectome_measure = ConnectivityMeasure(kind='correlation')
for filename, confound in zip(func_filenames, confounds):
    # call transform from RegionExtractor object to extract timeseries signals
    timeseries_each_subject = extractor.transform(filename, confounds=confound)
    # call fit_transform from ConnectivityMeasure object
    correlation = connectome_measure.fit_transform([timeseries_each_subject])
    # saving each subject correlation to correlations
    correlations.append(correlation)

# Mean of all correlations
# import numpy as np
mean_correlations = np.mean(correlations, axis=0).reshape(n_regions_extracted,
                                                          n_regions_extracted)

In [ ]:
title = 'Correlation between %d regions' % n_regions_extracted

# First plot the matrix
display = plotting.plot_matrix(mean_correlations, vmax=1, vmin=-1,
                               colorbar=True, title=title)

# Then find the center of the regions and plot a connectome
regions_img = regions_extracted_img
coords_connectome = plotting.find_probabilistic_atlas_cut_coords(regions_img)

plotting.plot_connectome(mean_correlations, coords_connectome,
                         edge_threshold='90%', title=title)

In [ ]:
# First, we plot a network of index=4 without region extraction (left plot)
# from nilearn import image

img = image.index_img(components_img, 4)
coords = plotting.find_xyz_cut_coords(img)
display = plotting.plot_stat_map(img, cut_coords=coords, colorbar=False,
                                 title='Showing one specific network')

In [ ]:
# For this, we take the indices of the all regions extracted related to original
# network given as 4.
regions_indices_of_map3 = np.where(np.array(regions_index) == 4)

display = plotting.plot_anat(cut_coords=coords,
                             title='Regions from this network')

# Add as an overlay all the regions of index 4
colors = 'rgbcmyk'
for each_index_of_map3, color in zip(regions_indices_of_map3[0], colors):
    display.add_overlay(image.index_img(regions_extracted_img, each_index_of_map3),
                        cmap=plotting.cm.alpha_cmap(color))

plotting.show()

In [ ]:
# from nilearn import datasets

# By default 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('First anatomical nifti image (3D) located is at: %s' %
      haxby_dataset.anat[0])
print('First functional nifti image (4D) is located at: %s' %
      haxby_dataset.func[0])

In [ ]:
from nilearn.image.image import mean_img

# Compute the mean EPI: we do the mean along the axis 3, which is time
func_filename = haxby_dataset.func[0]
mean_haxby = mean_img(func_filename)

from nilearn.plotting import plot_epi, show
plot_epi(mean_haxby)

In [ ]:
from nilearn.masking import compute_epi_mask
mask_img = compute_epi_mask(func_filename)

# Visualize it as an ROI
from nilearn.plotting import plot_roi
plot_roi(mask_img, mean_haxby)

In [ ]:
from nilearn.masking import apply_mask
masked_data = apply_mask(func_filename, mask_img)

# masked_data shape is (timepoints, voxels). We can plot the first 150
# timepoints from two voxels

# And now plot a few of these
# import matplotlib.pyplot as plt
plt.figure(figsize=(7, 5))
plt.plot(masked_data[:150, :2])
plt.xlabel('Time [TRs]', fontsize=16)
plt.ylabel('Intensity', fontsize=16)
plt.xlim(0, 150)
plt.subplots_adjust(bottom=.12, top=.95, right=.95, left=.12)

show()

In [ ]:
# from nilearn import datasets

motor_images = datasets.fetch_neurovault_motor_task()
stat_img = motor_images.images[0]

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage()

In [ ]:
from nilearn import surface

texture = surface.vol_to_surf(stat_img, fsaverage.pial_right)

In [ ]:
# from nilearn import plotting

plotting.plot_surf_stat_map(fsaverage.infl_right, texture, hemi='right',
                            title='Surface right hemisphere', colorbar=True,
                            threshold=1., bg_map=fsaverage.sulc_right)

In [ ]:
plotting.plot_glass_brain(stat_img, display_mode='r', plot_abs=False,
                          title='Glass brain', threshold=2.)

plotting.plot_stat_map(stat_img, display_mode='x', threshold=1.,
                       cut_coords=range(0, 51, 10), title='Slices')

In [ ]:
big_fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
big_texture = surface.vol_to_surf(stat_img, big_fsaverage.pial_right)

plotting.plot_surf_stat_map(big_fsaverage.infl_right,
                            big_texture, hemi='right', colorbar=True,
                            title='Surface right hemisphere: fine mesh',
                            threshold=1., bg_map=big_fsaverage.sulc_right)


plotting.show()

In [ ]:
view = plotting.view_surf(fsaverage.infl_right, texture, threshold='90%',
                          bg_map=fsaverage.sulc_right)

# In a Jupyter notebook, if ``view`` is the output of a cell, it will
# be displayed below the cell
view

In [ ]:
view = plotting.view_img_on_surf(stat_img, threshold='90%')
# view.open_in_browser()

view

In [ ]:
# from nilearn import datasets
print('Datasets are stored in: %r' % datasets.get_data_dirs())

In [ ]:
motor_images = datasets.fetch_neurovault_motor_task()
motor_images.images

In [ ]:
tmap_filename = motor_images.images[0]

# from nilearn import plotting
plotting.plot_stat_map(tmap_filename)

In [ ]:
plotting.plot_stat_map(tmap_filename, threshold=3)

In [ ]:
rsn = datasets.fetch_atlas_smith_2009()['rsn10']
rsn

In [ ]:
# from nilearn import image
print(image.load_img(rsn).shape)

In [ ]:
first_rsn = image.index_img(rsn, 0)
print(first_rsn.shape)

In [ ]:
plotting.plot_stat_map(first_rsn)

In [ ]:
for img in image.iter_img(rsn):
    # img is now an in-memory 3D img
    plotting.plot_stat_map(img, threshold=3, display_mode="z", cut_coords=1,
                           colorbar=False)

In [ ]:
selected_volumes = image.index_img(rsn, slice(3, 5))

In [ ]:
for img in image.iter_img(selected_volumes):
    plotting.plot_stat_map(img)

In [ ]:
# Load 4D probabilistic atlases
# from nilearn import datasets

# Harvard Oxford Atlasf
harvard_oxford = datasets.fetch_atlas_harvard_oxford('cort-prob-2mm')
harvard_oxford_sub = datasets.fetch_atlas_harvard_oxford('sub-prob-2mm')

# Multi Subject Dictionary Learning Atlas
msdl = datasets.fetch_atlas_msdl()

# Smith ICA Atlas and Brain Maps 2009
smith = datasets.fetch_atlas_smith_2009()

# ICBM tissue probability
icbm = datasets.fetch_icbm152_2009()

# Allen RSN networks
allen = datasets.fetch_atlas_allen_2011()

# Pauli subcortical atlas
subcortex = datasets.fetch_atlas_pauli_2017()

# Visualization
# from nilearn import plotting

atlas_types = {'Harvard_Oxford': harvard_oxford.maps,
               'Harvard_Oxford sub': harvard_oxford_sub.maps,
               'MSDL': msdl.maps, 'Smith 2009 10 RSNs': smith.rsn10,
               'Smith2009 20 RSNs': smith.rsn20,
               'Smith2009 70 RSNs': smith.rsn70,
               'Smith2009 20 Brainmap': smith.bm20,
               'Smith2009 70 Brainmap': smith.bm70,
               'ICBM tissues': (icbm['wm'], icbm['gm'], icbm['csf']),
               'Allen2011': allen.rsn28,
               'Pauli2017 Subcortical Atlas': subcortex.maps,
               }

for name, atlas in sorted(atlas_types.items()):
    plotting.plot_prob_atlas(atlas, title=name)

# An optional colorbar can be set
plotting.plot_prob_atlas(smith.bm10, title='Smith2009 10 Brainmap (with'
                                           ' colorbar)',
                         colorbar=True)
print('ready')
plotting.show()

# EDA 

In [ ]:
!pip install joypy --progress-bar off

In [ ]:
import scipy as sp
import random
import tensorflow as tf
import cv2
# General packages
import PIL
import plotly.graph_objs as go
from IPython.display import Image, display
import joypy

In [ ]:
os.listdir('/kaggle/input/trends-assessment-prediction/')

In [ ]:
BASE_PATH = '../input/trends-assessment-prediction'

# image and mask directories
train_data_dir = f'{BASE_PATH}/fMRI_train'
test_data_dir = f'{BASE_PATH}/fMRI_test'


print('Reading data...')
loading_data = pd.read_csv(f'{BASE_PATH}/loading.csv')
train_data = pd.read_csv(f'{BASE_PATH}/train_scores.csv')
sample_submission = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')
print('Reading data completed')

In [ ]:
display(train_data.head())
print("Shape of train_data :", train_data.shape)

In [ ]:
display(loading_data.head())
print("Shape of loading_data :", loading_data.shape)

In [ ]:
# checking missing data
total = train_data.isnull().sum().sort_values(ascending = False)
percent = (train_data.isnull().sum()/train_data.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_data.head()

In [ ]:
total = loading_data.isnull().sum().sort_values(ascending = False)
percent = (loading_data.isnull().sum()/loading_data.isnull().count()*100).sort_values(ascending = False)
missing_loading_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_loading_data.head()

In [ ]:
def plot_bar(df, feature, title='', show_percent = False, size=2):
    f, ax = plt.subplots(1,1, figsize=(4*size,3*size))
    total = float(len(df))
    sns.barplot(np.round(df[feature].value_counts().index).astype(int), df[feature].value_counts().values, alpha=0.8, palette='Set2')

    plt.title(title)
    if show_percent:
        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x()+p.get_width()/2.,
                    height + 3,
                    '{:1.2f}%'.format(100*height/total),
                    ha="center", rotation=45) 
    plt.xlabel(feature, fontsize=12, )
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
plot_bar(train_data, 'age', 'age count and %age plot', show_percent=True, size=4)

In [ ]:
def plot_bar(df, feature, title='', show_percent = False, size=2):
    f, ax = plt.subplots(1,1, figsize=(4*size,3*size))
    total = float(len(df))
    sns.barplot(np.round(df[feature].value_counts().index).astype(int), df[feature].value_counts().values, alpha=0.8, palette='Set2')

    plt.title(title)
    if show_percent:
        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x()+p.get_width()/2.,
                    height + 3,
                    '{:1.2f}%'.format(100*height/total),
                    ha="center", rotation=45) 
    plt.xlabel(feature, fontsize=12, )
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
### Age count Distribution
for col in train_data.columns[2:]:
    plot_bar(train_data, col, f'{col} count plot', size=4)

In [ ]:
temp_data =  train_data.drop(['Id'], axis=1)

plt.figure(figsize = (12, 8))
sns.heatmap(temp_data.corr(), annot = True, cmap="RdYlGn")
plt.yticks(rotation=0) 

plt.show()

In [ ]:
temp_data =  loading_data.drop(['Id'], axis=1)

plt.figure(figsize = (20, 20))
sns.heatmap(temp_data.corr(), annot = True, cmap="RdYlGn")
plt.yticks(rotation=0) 

plt.show()

In [ ]:
temp_data =  loading_data.drop(['Id'], axis=1)
# Create correlation matrix
correl = temp_data.corr().abs()

# Select upper triangle of correlation matrix
upper = correl.where(np.triu(np.ones(correl.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.5
to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]

print('Very high correlated features: ', to_drop)

In [ ]:
# Draw Plot
import joypy

targets= loading_data.columns[1:]


plt.figure(figsize=(16,10), dpi= 80)
fig, axes = joypy.joyplot(loading_data, column=list(targets), ylim='own', figsize=(14,10))

# Decoration
plt.title('Distribution of features IC_01 to IC_29', fontsize=22)
plt.show()

In [ ]:
"""
    Load and display a subject's spatial map
"""

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
mask_filename = f'{BASE_PATH}/fMRI_mask.nii'
subject_filename = '../input/trends-assessment-prediction/fMRI_train/10004.mat'
smri_filename = 'ch2better.nii'
mask_niimg = nl.image.load_img(mask_filename)


def load_subject(filename, mask_niimg):
    """
    Load a subject saved in .mat format with
        the version 7.3 flag. Return the subject
        niimg, using a mask niimg as a template
        for nifti headers.
        
    Args:
        filename    <str>            the .mat filename for the subject data
        mask_niimg  niimg object     the mask niimg object used for nifti headers
    """
    subject_data = None
    with h5py.File(subject_filename, 'r') as f:
        subject_data = f['SM_feature'][()]
    # It's necessary to reorient the axes, since h5py flips axis order
    subject_data = np.moveaxis(subject_data, [0,1,2,3], [3,2,1,0])
    subject_niimg = nl.image.new_img_like(mask_niimg, subject_data, affine=mask_niimg.affine, copy_header=True)
    return subject_niimg
subject_niimg = load_subject(subject_filename, mask_niimg)
print("Image shape is %s" % (str(subject_niimg.shape)))
num_components = subject_niimg.shape[-1]
print("Detected {num_components} spatial maps".format(num_components=num_components))

In [ ]:
nlplt.plot_prob_atlas(subject_niimg, bg_img=smri_filename, view_type='filled_contours', draw_cross=False,title='All %d spatial maps' % num_components, threshold='auto')

In [ ]:
grid_size = int(np.ceil(np.sqrt(num_components)))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size*10, grid_size*10))
[axi.set_axis_off() for axi in axes.ravel()]
row = -1
for i, cur_img in enumerate(nl.image.iter_img(subject_niimg)):
    col = i % grid_size
    if col == 0:
        row += 1
    nlplt.plot_stat_map(cur_img, bg_img=smri_filename, title="IC %d" % i, axes=axes[row, col], threshold=3, colorbar=False)

## Sample Submission

In [ ]:

!pip install pycaret --quiet

In [ ]:
from pycaret.regression import *

In [ ]:
BASE_PATH = '../input/trends-assessment-prediction'

fnc_df = pd.read_csv(f"{BASE_PATH}/fnc.csv")
loading_df = pd.read_csv(f"{BASE_PATH}/loading.csv")
labels_df = pd.read_csv(f"{BASE_PATH}/train_scores.csv")

In [ ]:
fnc_features, loading_features = list(fnc_df.columns[1:]), list(loading_df.columns[1:])
df = fnc_df.merge(loading_df, on="Id")
labels_df["is_train"] = True
df = df.merge(labels_df, on="Id", how="left")

test_df = df[df["is_train"] != True].copy()

In [ ]:
target_cols = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
test_df = test_df.drop(target_cols + ['is_train'], axis=1)

# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/500
test_df[fnc_features] *= FNC_SCALE

In [ ]:
target_models_dict = {
    'age': 'age_br',
    'domain1_var1':'domain1_var1_ridge',
    'domain1_var2':'domain1_var2_svm',
    'domain2_var1':'domain2_var1_ridge',
    'domain2_var2':'domain2_var2_svm',
}

In [ ]:
from sklearn.utils.fixes import logsumexp 

In [ ]:

for index, target in enumerate(target_cols):
    model_name = target_models_dict[target]
    model = load_model(f'../input/pycaret-trends-models/{model_name}', platform = None, authentication = None, verbose=True)

    predictions = predict_model(model, data=test_df)
    test_df[target] = predictions['Label'].values

### Create Submission

In [ ]:
sub_df = pd.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5

sub_df.to_csv("submission1.csv", index=False)
sub_df.head()

# Further Analysis

In [ ]:
features = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv')
loading = pd.read_csv('/kaggle/input/trends-assessment-prediction/loading.csv')
submission = pd.read_csv('/kaggle/input/trends-assessment-prediction/sample_submission.csv')
fnc = pd.read_csv("/kaggle/input/trends-assessment-prediction/fnc.csv")
reveal = pd.read_csv('../input/trends-assessment-prediction/reveal_ID_site2.csv')
numbers = pd.read_csv('../input/trends-assessment-prediction/ICN_numbers.csv')
fmri_mask = '../input/trends-assessment-prediction/fMRI_mask.nii'

In [ ]:
smri = 'ch2better.nii'
mask_img = nl.image.load_img(fmri_mask)

def load_subject(filename, mask_img):
    subject_data = None
    with h5py.File(filename, 'r') as f:
        subject_data = f['SM_feature'][()]
    # It's necessary to reorient the axes, since h5py flips axis order
    subject_data = np.moveaxis(subject_data, [0,1,2,3], [3,2,1,0])
    subject_img = nl.image.new_img_like(mask_img, subject_data, affine=mask_img.affine, copy_header=True)

    return subject_img

# Plotting 4D probabilistic atlas maps...
Probabilistic atlasing is a research strategy whose goal is to generate anatomical templates that retain quantitative information on inter-subject variations in brain architecture (Mazziotta et al., 1995). A digital probabilistic atlas of the human brain, incorporating precise statistical information on positional variability of important functional and anatomic interfaces, may rectify many current atlasing problems, since it specifically stores information on the population variability.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    nlplt.plot_prob_atlas(subject_img, bg_img=smri, view_type='filled_contours',
                          draw_cross=False, title='All %d spatial maps' % num_components, threshold='auto')
    print("-"*50)

## Plotting a statistical map
Statistical parametric mapping or SPM is a statistical technique for examining differences in brain activity recorded during functional neuroimaging experiments.The measurement technique depends on the imaging technology (e.g., fMRI and PET). The scanner produces a 'map' of the area that is represented as voxels. Each voxel represents the activity of a specific volume in three-dimensional space. The exact size of a voxel varies depending on the technology. fMRI voxels typically represent a volume of 27 mm3 (a cube with 3mm length sides).

Parametric statistical models are assumed at each voxel, using the general linear model to describe the data variability in terms of experimental and confounding effects, with residual variability. Hypotheses expressed in terms of the model parameters are assessed at each voxel with univariate statistics.

Analyses may examine differences over time (i.e. correlations between a task variable and brain activity in a certain area) using linear convolution models of how the measured signal is caused by underlying changes in neural activity.

Because many statistical tests are conducted, adjustments have to be made to control for type I errors (false positives) potentially caused by the comparison of levels of activity over many voxels. A type I error would result in falsely assessing background brain activity as related to the task. Adjustments are made based on the number of resels in the image and the theory of continuous random fields in order to set a new criterion for statistical significance that adjusts for the problem of multiple comparisons.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)
    plotting.plot_stat_map(first_rsn)
    print("-"*50)

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)
    for img in image.iter_img(rsn):
        # img is now an in-memory 3D img
        plotting.plot_stat_map(img, threshold=3)
    print("-"*50)

### Glass brain visualization...
Glass Brain is a tool that maps the electrical activity of your brain in realtime.The anatomically realistic 3D brain will show realtime data from electroencephalographic (EEG) signals taken from a specially-designed EEG cap.This data is mapped to the source of that electrical activity, i.e. the specific part of the brain. The underlying brain model is generated through MRI scans so that the EEG data is accurately mapped to an individual's brain model.

Different colours are given to the different signal frequency bands to create a beautiful interactive artwork that seems to crackle with energy, showing how information is transferred (or at least estimated to do so) between different regions of the brain.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)     
    plotting.plot_glass_brain(first_rsn,display_mode='lyrz')
    print("-"*50)

## Plotting an EPI
In Echo-Planar Imaging (EPI)-based Magnetic Resonance Imaging (MRI), inter-subject registration typically uses the subject's T1-weighted (T1w) anatomical image to learn deformations of the subject's brain onto a template. The estimated deformation fields are then applied to the subject's EPI scans (functional or diffusion-weighted images) to warp the latter to a template space.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)
    plotting.plot_epi(first_rsn)
    print("-"*50)

## Plotting an anatomical image
Main Idea of this visualization technique is to provide the anatomical picture of the brain. Due to the measurement procedures, BOLD images usually have a realtively low resolution, as you want to squeeze in as many data-points along time as possible.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)
    plotting.plot_anat(first_rsn)
    print("-"*50)

## Plotting ROIs, or a mask
when mapping brain connectivities, ROIs provide the structural substrates for measuring connectivities within individual brains and for pooling data across populations. Thus, identification of reliable, reproducible and accurate ROIs is critically important for the success of brain connectivity mapping.

In [ ]:
files = random.choices(os.listdir('../input/trends-assessment-prediction/fMRI_train/'), k = 3)
for file in files:
    subject = os.path.join('../input/trends-assessment-prediction/fMRI_train/', file)
    subject_img = load_subject(subject, mask_img)
    print("Image shape is %s" % (str(subject_img.shape)))
    num_components = subject_img.shape[-1]
    print("Detected {num_components} spatial maps".format(num_components=num_components))
    rsn = subject_img
    #convert to 3d image
    first_rsn = image.index_img(rsn, 0)
    print(first_rsn.shape)
    plotting.plot_roi(first_rsn)
    print("-"*50)

## 3D Plots of statistical maps or atlases on the cortical surface

In [ ]:
motor_images = datasets.fetch_neurovault_motor_task()
stat_img = motor_images.images[0]
view = plotting.view_img_on_surf(stat_img, threshold='90%')
view.open_in_browser()
view

In [ ]:
features.head()

In [ ]:
features.info()

#### WHAT DO WE DO TO MISSING VALUES?
There are several options for handling missing values each with its own PROS and CONS. However, the choice of what should be done is largely dependent on the nature of our data and the missing values. Below is a summary highlight of several options we have for handling missing values.

#### DROP MISSING VALUES
FILL MISSING VALUES WITH TEST STATISTIC(mean, median, mode).
PREDICT MISSING VALUE WITH A MACHINE LEARNING ALGORITHM(knn).

In [ ]:
features.fillna(features.mean(),inplace=True)

In [ ]:
features.info()

In [ ]:
loading.head()

In [ ]:
loading.info()

In [ ]:
fnc.head()

### Checking the correlation between features

In [ ]:
sns.heatmap(features.corr(),annot=True,linewidths=0.2) 
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
sns.heatmap(loading.corr(),annot=True,linewidths=0.2) 
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
#main or, target element in problem
target_col = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

#### Visualize the Target Columns

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 5))
sns.distplot(features['age'], ax=ax[0],rug=True, rug_kws={"color": "coral"},
                  kde_kws={"color": "royalblue", "lw": 1.5},
                  hist_kws={"histtype": "bar", "linewidth": 3,
                            "alpha": 1, "color": "coral"}).set_title('Age')

sns.distplot(features['domain1_var1'], ax=ax[1],rug=True, rug_kws={"color": "coral"},
                  kde_kws={"color": "royalblue", "lw": 1.5},
                  hist_kws={"histtype": "bar", "linewidth": 3,
                            "alpha": 1, "color": "coral"}).set_title('domain1_var1')

sns.distplot(features['domain1_var2'], ax=ax[2],rug=True, rug_kws={"color": "coral"},
                  kde_kws={"color": "royalblue", "lw": 1.5},
                  hist_kws={"histtype": "bar", "linewidth": 3,
                            "alpha": 1, "color": "coral"}).set_title('domain1_var2')

sns.distplot(features['domain2_var1'], ax=ax[3],rug=True, rug_kws={"color": "coral"},
                  kde_kws={"color": "royalblue", "lw": 1.5},
                  hist_kws={"histtype": "bar", "linewidth": 3,
                            "alpha": 1, "color": "coral"}).set_title('domain2_var1')

sns.distplot(features['domain2_var2'], ax=ax[4],rug=True, rug_kws={"color": "coral"},
                  kde_kws={"color": "royalblue", "lw": 1.5},
                  hist_kws={"histtype": "bar", "linewidth": 3,
                            "alpha": 1, "color": "coral"}).set_title('domain2_var2')

fig.suptitle('Target Visualization', fontsize=10)

In [ ]:
plt.figure(figsize=(20,15))
g = sns.pairplot(data=features, hue='age', palette = 'seismic',
                 size=1.2,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])

# Again, 
# Model using Rapids for 500x faster kNN on GPUs

## Install Rapids for 500x faster kNN on GPUs

In [ ]:
import sys
!cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import numpy as np
import pandas as pd
import cudf
import cupy as cp
from cuml.neighbors import KNeighborsRegressor
from cuml import SVR
from cuml.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from cuml.metrics import mean_absolute_error, mean_squared_error


def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [ ]:
fnc_df = cudf.read_csv("../input/trends-assessment-prediction/fnc.csv")
loading_df = cudf.read_csv("../input/trends-assessment-prediction/loading.csv")

fnc_features, loading_features = list(fnc_df.columns[1:]), list(loading_df.columns[1:])
df = fnc_df.merge(loading_df, on="Id")


labels_df = cudf.read_csv("../input/trends-assessment-prediction/train_scores.csv")
labels_df["is_train"] = True

df = df.merge(labels_df, on="Id", how="left")

test_df = df[df["is_train"] != True].copy()
df = df[df["is_train"] == True].copy()

df.shape, test_df.shape

In [ ]:
# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/600

df[fnc_features] *= FNC_SCALE
test_df[fnc_features] *= FNC_SCALE

In [ ]:
%%time

NUM_FOLDS = 7
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)


features = loading_features + fnc_features

overal_score = 0
for target, c, w, ff in [("age", 60, 0.3, 0.55), ("domain1_var1", 12, 0.175, 0.2), ("domain1_var2", 8, 0.175, 0.2), ("domain2_var1", 9, 0.175, 0.22), ("domain2_var2", 12, 0.175, 0.22)]:    
    y_oof = np.zeros(df.shape[0])
    y_test = np.zeros((test_df.shape[0], NUM_FOLDS))

    
    for f, (train_ind, val_ind) in enumerate(kf.split(df, df)):
        train_df, val_df = df.iloc[train_ind], df.iloc[val_ind]
        train_df = train_df[train_df[target].notnull()]

        model_1 = SVR(C=c, cache_size=3000.0)
        model_1.fit(train_df[features].values, train_df[target].values)
        model_2 = Ridge(alpha = 0.0001)
        model_2.fit(train_df[features].values, train_df[target].values)
        val_pred_1 = model_1.predict(val_df[features])
        val_pred_2 = model_2.predict(val_df[features])
        
        test_pred_1 = model_1.predict(test_df[features])
        test_pred_2 = model_2.predict(test_df[features])
        
        val_pred = (1-ff)*val_pred_1+ff*val_pred_2
        val_pred = cp.asnumpy(val_pred.values.flatten())
        
        test_pred = (1-ff)*test_pred_1+ff*test_pred_2
        test_pred = cp.asnumpy(test_pred.values.flatten())

        y_oof[val_ind] = val_pred
        y_test[:, f] = test_pred
        
    df["pred_{}".format(target)] = y_oof
    test_df[target] = y_test.mean(axis=1)
    
    score = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_{}".format(target)].values)
    mae = mean_absolute_error(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_{}".format(target)].values)
    rmse = np.sqrt(mean_squared_error(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_{}".format(target)].values))
    overal_score += w*score
    print(target, np.round(score, 8))
    print(target, np.round(score, 4))
    print(target, 'mean absolute error:', np.round(mae, 8))
    print(target, ' root mean square error:', np.round(rmse, 8))
    print()
    
print("Overal score:", np.round(overal_score, 8))
print("Overal score:", np.round(overal_score, 4))

In [ ]:
sub_df = cudf.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.head(10)

In [ ]:
sub_df.to_csv("submission_rapids_ensemble.csv", index=False)